In [14]:
## IMPORTS
import random
from deap import base
from deap import creator
from deap import tools
import numpy as np

In [15]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

/opt/anaconda3/envs/titanic/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/opt/anaconda3/envs/titanic/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [16]:
def generate_random_num_attr():
  original = np.array([0.8, 0.3,0.1, 1.03, 1.1, 1.05, 0.975, 1])
  noises = []
  sum_list = []

  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.01))
  noises.append(np.random.normal(0, 0.001))
  noises.append(np.random.normal(0, 0.01))
  noises.append(np.random.normal(0, 0.005))
  noises.append(np.random.normal(0, 0.005))
  noises.append(np.random.normal(0, 0.01))

  for (item1, item2) in zip(original, noises):
      sum_list.append(item1+item2)
  
  return sum_list

In [17]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", generate_random_num_attr)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [18]:
## change to our functinon
from stage import getStage
import numpy as np
import pandas as pd
from datetime import date
import yahoo_fin.stock_info as yf
def evalOneMax(individual):
    print(individual)
    transaction = yf.get_data('AAPL', start_date="1995-01-02",end_date="2022-01-09", index_as_date = True, interval="1wk").drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    for symbol in yf.tickers_sp500():
        df = getStage(symbol,individual)
        inStage = False
        buyTwice = False
        if df.empty:
            continue
        for index, element in df.iterrows():
            index = index.strftime('%Y-%m-%d')
            if element.Stage == "Stage 2" or element.Stage == "Buy":
                transaction.loc[index]['holding'].append((symbol,element.close,0))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                if element.Stage == "Buy" and inStage:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                    buyTwice = True
                inStage = True
                continue
            if element.Stage == "Sell":
                transaction.loc[index]['holding'].append((symbol,element.close,-1))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,-1))
                inStage = False
                buyTwice = False
    total = 100
    prevIndex = ''
    first = True
    transactionCopy = transaction
    transactionCopy['total'] = 0
    for index, element in transactionCopy.iterrows():
        index = index.strftime('%Y-%m-%d')
        if len(element.holding) == 0:
            transactionCopy['total'][index] = total
            continue
        if first:
            first = False
            transactionCopy['total'][index] = total
        else:
            prevData = transactionCopy.loc[prevIndex].holding
            if len(prevData):
                total = 0
                removeList = []
                close = 0
                for i in range(len(prevData)):
                    for j in range(len(element.holding)):
                        if element.holding[j][0] == prevData[i][0]:
                            close = element.holding[j][1]
                            if element.holding[j][2] == -1:
                                removeList.append(element.holding[j])
                            break
                    total += close*prevData[i][2]
                transactionCopy['total'][index] = total
                for delete in removeList:
                    if delete in element.holding:
                        element.holding.remove(delete)
            else:
                transactionCopy['total'][index] = transactionCopy.loc[prevIndex].total
        if len(element.holding):
            allocation = total/len(element.holding)
        for i in range(len(element.holding)):
            element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
        prevIndex = index
    print(transactionCopy.iloc[-1]['total'])
    return transactionCopy.iloc[-1]['total']

In [19]:
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [20]:
def main():
    pop = toolbox.population(n=20)
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
        
    # Begin the evolution
    for g in range(10):
        print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.5:
                toolbox.mate(child1[0], child2[0])
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < 0.2:
                toolbox.mutate(mutant[0])
                del mutant.fitness.values        
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        # Replace population
        pop[:] = offspring
    
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

In [21]:
main()

[[0.8415825488719344, 1.8820801790078399, 0.09344664612828468, 1.028971310264687, 1.1025183282185094, 1.043771099028405, 0.9699446362959303, 0.9998396879884038]]
